# February 06, 2024 - Session Notes

1. Multi Route Chains

In [34]:
from langchain.chains import LLMChain
from langchain.chains.router import RouterChain, MultiRouteChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
model = ChatGoogleGenerativeAI(model='gemini-pro')

In [15]:
scientist = '''
You are a scientist.
You do not answer any question that is outside the fold of science concepts.
Here is the question for you. \n {input}
'''

mathematician = '''
You are a mathematician.
You do not answer any question that is outside the fold of mathematic concepts.
Here is the question for you. \n {input}
'''

lawyer = '''
You are a Lawyer. 
You do not answer any question that is outside the fold of law concepts.
Here is the question for you. \n {input}
'''

In [16]:
prompt_infos = [
    {
        "name" : "scientist",
        "desc" : "Rsponds to science questions only",
        "prompt_template" : scientist
    },
    {
        "name" : "mathematician",
        "desc" : "Responds to mathematics questions only",
        "prompt_template" : mathematician
    },
    {
        "name" : "Lawyer",
        "desc" : "Responds to law questions only",
        "prompt_template" : lawyer
    }
]

In [17]:
default_prompt = ChatPromptTemplate.from_template(template='{input}')
default_chain = LLMChain(llm=model, prompt=default_prompt)

In [18]:
destination_chains = {}

for _ in prompt_infos:
    name = _['name']
    templ = _['prompt_template']
    chat_prompt = ChatPromptTemplate.from_template(templ)
    chain = LLMChain(llm=model, prompt=chat_prompt)
    destination_chains[name] = chain

In [19]:
destination_chains

{'scientist': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='\nYou are a scientist.\nYou do not answer any question that is outside the fold of science concepts.\nHere is the question for you. \n {input}\n'))]), llm=ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={}.
    safety_settings={}
 ))),
 'mathematician': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='\nYou are a mathematician.\nYou do not answer any question that is outside the fold of mathematic concepts.\nHere is the question for you. \n {input}\n'))]), llm=ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={}.
    safety_settings={}
 ))),
 'Law

In [24]:
#routing destinations

destinations = [f"{_['name']} : {_['desc']}" for _ in prompt_infos]

In [25]:
destinations

['scientist : Rsponds to science questions only',
 'mathematician : Responds to mathematics questions only',
 'Lawyer : Responds to law questions only']

In [26]:
destinations_str = '\n'.join(destinations)

In [28]:
router_temp = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

In [31]:
print(router_temp)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
scientist : Rsponds to science questions only
mathematician : Responds to mathematics questions only


In [45]:
router_prompt = PromptTemplate(
    template=router_temp, 
    input_variables=['input'], 
    output_parser=RouterOutputParser()
    )

router_chain = LLMRouterChain.from_llm(llm=model, prompt=router_prompt)

In [49]:
chain = MultiRouteChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True )

In [59]:
result = chain.invoke("What is an atom?")



> Entering new MultiRouteChain chain...


C:\Users\mfurq\AppData\Roaming\Python\Python311\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


scientist: {'input': 'What is an atom?'}
> Finished chain.


In [60]:
print(result['text'])

An atom is the basic unit of matter and the defining structure of elements. Atoms are composed of three types of subatomic particles: protons, neutrons, and electrons. Protons and neutrons are found in the nucleus of an atom, while electrons orbit the nucleus. The number of protons in an atom determines the element to which it belongs. For example, all atoms with one proton are hydrogen atoms, all atoms with two protons are helium atoms, and so on. The number of neutrons in an atom can vary, resulting in different isotopes of the same element. For example, carbon-12, carbon-13, and carbon-14 are all isotopes of carbon, each with a different number of neutrons. Atoms can combine with each other to form molecules, which are the building blocks of all matter.


<<< End Of Document >>>